## Web scraping and analysis

This Jupyter notebook includes code for web scraping the reviews of British Airways. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [9]:
base_AirlineReviews_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 38
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_AirlineReviews_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("article", {"itemprop": "review"}):
        temp = {}
        #print(para)
        
        # Overall Rating
        try:
            temp["OverallRating"] = para.find_all("span", {"itemprop":"ratingValue"})[0].get_text().lstrip()
        except:
            temp["OverallRating"] = None
        
        # Review Header
        temp["ReviewHeader"] = para.find_all("h2", {"class":"text_header"})[0].get_text().lstrip()
        
        # Reviewer Name
        temp["Name"] = para.find_all("span", {"itemprop":"name"})[0].get_text().lstrip()
        
        # Review Date Published
        temp["Datetime"] = para.find_all("time", {"itemprop":"datePublished"})[0].get_text().lstrip()
        
        # Review Body
        try:
            isVerivied, reviewBody = para.find_all("div", {"itemprop":"reviewBody"})[0].get_text().split("|")
        except:
            isVerivied = "unknown"
            reviewBody = para.find_all("div", {"itemprop":"reviewBody"})[0].get_text()
        temp["VerifiedReview"] = True if isVerivied.lstrip().rstrip() == "✅ Trip Verified" else False
        temp["ReviewBody"] = reviewBody.lstrip()
        
        # Table Data
        for tr in para.find_all("tr"):
            trContent = tr.find_all("td")
            clmHeader = trContent[0].get_text().replace(' ', '')
            clmContent= trContent[1]
            if clmContent.find_all("span", {"class":"fill"}):
                clmData = clmContent.find_all("span", {"class":"fill"})[-1].get_text().lstrip()
            else:
                clmData = clmContent.get_text().lstrip()
            temp[clmHeader] = clmData
        
        reviews.append(temp)
    
    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1
   ---> 100 total reviews
Scraping page 2
   ---> 200 total reviews
Scraping page 3
   ---> 300 total reviews
Scraping page 4
   ---> 400 total reviews
Scraping page 5
   ---> 500 total reviews
Scraping page 6
   ---> 600 total reviews
Scraping page 7
   ---> 700 total reviews
Scraping page 8
   ---> 800 total reviews
Scraping page 9
   ---> 900 total reviews
Scraping page 10
   ---> 1000 total reviews
Scraping page 11
   ---> 1100 total reviews
Scraping page 12
   ---> 1200 total reviews
Scraping page 13
   ---> 1300 total reviews
Scraping page 14
   ---> 1400 total reviews
Scraping page 15
   ---> 1500 total reviews
Scraping page 16
   ---> 1600 total reviews
Scraping page 17
   ---> 1700 total reviews
Scraping page 18
   ---> 1800 total reviews
Scraping page 19
   ---> 1900 total reviews
Scraping page 20
   ---> 2000 total reviews
Scraping page 21
   ---> 2100 total reviews
Scraping page 22
   ---> 2200 total reviews
Scraping page 23
   ---> 2300 total reviews
Scrapi

In [12]:
df = pd.DataFrame(reviews)
df.head()

,OverallRating,ReviewHeader,Name,Datetime,VerifiedReview,ReviewBody,TypeOfTraveller,SeatType,Route,DateFlown,SeatComfort,CabinStaffService,GroundService,ValueForMoney,Recommended,Aircraft,Food&Beverages,InflightEntertainment,Wifi&Connectivity
0,1,"""Service level far worse then Ryanair""",L Keele,19th November 2023,True,4 Hours before takeoff we received a Mail stat...,Couple Leisure,Economy Class,London to Stuttgart,November 2023,1,1,1,1,no,NaN,NaN,NaN,NaN
1,3,"""do not upgrade members based on status""",Austin Jones,19th November 2023,True,I recently had a delay on British Airways from...,Business,Economy Class,Brussels to London,November 2023,2,3,1,2,no,A320,1,2,2
2,8,"""Flight was smooth and quick""",M A Collie,16th November 2023,False,"Boarded on time, but it took ages to get to th...",Couple Leisure,Business Class,London Heathrow to Dublin,November 2023,3,3,4,3,yes,A320,4,NaN,NaN
3,1,"""Absolutely hopeless airline""",Nigel Dean,16th November 2023,True,"5 days before the flight, we were advised by B...",Couple Leisure,Economy Class,London to Dublin,December 2022,3,3,1,1,no,NaN,NaN,NaN,NaN
4,1,"""Customer Service is non existent""",Gaylynne Simpson,14th November 2023,False,"We traveled to Lisbon for our dream vacation, ...",Couple Leisure,Economy Class,London to Lisbon,November 2023,1,1,1,1,no,NaN,1,1,1


In [14]:
df.shape

(3701, 19)

In [11]:
df.to_csv("Dataset/BA_AirlineReviews.csv")

The Data is now collected and stored in a CSV file. The data can now be used to do further analysis.